In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install trl
!pip install streamlit
!pip install youtube_comment_downloader
!pip install streamlit_option_menu
!pip install streamlit_player
!npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 2.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import login
login()

In [ ]:
%%writefile app.py
import streamlit as st
import pandas as pd
from itertools import islice
from PIL import Image
from youtube_comment_downloader import *
from tkinter.tix import COLUMN
from pyparsing import empty
from streamlit_option_menu import option_menu
from streamlit_player import st_player
from transformers import GPT2LMHeadModel, GPT2Tokenizer

import os
from datasets import load_dataset

# Set Cache Dir
CACHE_DIR = os.getcwd()+'/cache'

import torch
from accelerate import Accelerator
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer
from peft import LoraConfig, get_peft_model
from transformers import TrainingArguments
from trl import SFTTrainer
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig
from transformers import TextStreamer, GenerationConfig
import torch
import re

# 업로드된 파일에서 혐오 단어 목록을 읽기
file_name = "/content/korean-bad-words.txt" # 첫 번째 업로드된 파일 이름
with open(file_name, 'r', encoding='utf-8') as file:
    bad_words = file.readlines()
bad_words = [word.strip() for word in bad_words]

# 혐오 단어를 찾아 '*'로 마스킹하는 최적화된 함수
def mask_bad_words_optimized(sentence, bad_words):
    pattern = '|'.join(map(re.escape, bad_words))
    return re.sub(pattern, lambda x: '*' * len(x.group()), sentence)


model_id2 = "maywell/Synatra-42dot-1.3B"  #분류모델
model_name = "beomi/Yi-Ko-6B" #beomi/OPEN-SOLAR-KO-10.7B" #변환모델

#분류모델 먼저

bnb_config10 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer2 = AutoTokenizer.from_pretrained(model_id2)
model2 = AutoModelForCausalLM.from_pretrained(model_id2, quantization_config=bnb_config10, device_map={"":0})

peft_model_name10 = '/content/drive/MyDrive/Colab Notebooks/국립국어원 혐오표현/model/outputs'
config10 = PeftConfig.from_pretrained(peft_model_name10)
    # model_name="beomi/Yi-Ko-6B"
    # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
bnb_config10 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
        )
config10.base_model_name_or_path =model2

model2 = PeftModel.from_pretrained(model2, peft_model_name10)

def gen(x):
    prompt = f"### 문장: {x}\n\n### 유형:"
    gened = model2.generate(
        **tokenizer2(
            prompt,
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=2,
        early_stopping=True,
        do_sample=True,
        eos_token_id=50256,
    )
    return "혐오" if (tokenizer2.decode(gened[0]).replace(prompt+ " ", "")=="<||bos||>혐오") else "비혐오"

# 여기부터 변환모델
# Quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load Model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    quantization_config=bnb_config,
    device_map={"": Accelerator().local_process_index},
    use_auth_token=True,
    trust_remote_code=True,
    cache_dir=CACHE_DIR
)
model.config.use_cache = False

# Set-Up Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, cache_dir=CACHE_DIR)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

peft_model_name = '/content/drive/MyDrive/Colab Notebooks/국립국어원 혐오표현/model/checkpoint-2000'
config = PeftConfig.from_pretrained(peft_model_name)
    # model_name="beomi/Yi-Ko-6B"
    # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
        )
config.base_model_name_or_path =model

model = PeftModel.from_pretrained(model, peft_model_name)

peft_model_name2 = '/content/drive/MyDrive/Colab Notebooks/국립국어원 혐오표현/model/checkpoint-1000'
config2 = PeftConfig.from_pretrained(peft_model_name2)
    # model_name="beomi/Yi-Ko-6B"
    # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
bnb_config2 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
        )
config2.base_model_name_or_path =model

model = PeftModel.from_pretrained(model, peft_model_name2)

peft_model_name3 = '/content/drive/MyDrive/Colab Notebooks/국립국어원 혐오표현/model/finalgen2'
config3 = PeftConfig.from_pretrained(peft_model_name3)
    # model_name="beomi/Yi-Ko-6B"
    # model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
bnb_config3 = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16
        )
config3.base_model_name_or_path =model

model = PeftModel.from_pretrained(model, peft_model_name3)

from transformers import TextStreamer, GenerationConfig
streamer = TextStreamer(tokenizer)
def gen2(x):
    generation_config = GenerationConfig(
        temperature=1,
        top_p=0.8,
        top_k=100,
        max_new_tokens=70,
        early_stopping=True,
        do_sample=True,
    )
    q = f"Human: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '보여주면서 왜 엿보냐고 비난 하는것도 웃기지만. 훔쳐 보면서 왜 보여주냐고 하는 사람 역시 우습다..' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '보여주면서 왜 엿보냐고 비난하는 것도 우습고, 훔쳐 보면서 왜 보여주냐고 하는 사람도 우습습니다.'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: 보여주는 것과 엿보는 행위는 서로 다른 문제이며, 각각에 대해 적절한 방식으로 대응해야 합니다.\nHuman: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '이런 쓰레기같은 새끼가 아무렇지 않게 멀쩡히 돌아다닐 생각을 하니까 진짜 너무 소름돋는다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '이런 나쁜 사람이 아무렇지 않게 돌아다니는 것을 생각하면 정말 소름이 돋습니다.'\nHuman: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '인간의 탈을 쓰고...' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '인간이라면 이럴 수 없어요.'\nHuman: 당신은 비속어나 혐오 표현을 받으면 올바르고 예의바른 문장으로 수정하는 예의바른 언어학자입니다. 사용자가 질문을 합니다. 당신의 임무는 가능한 한 충실하고 예의바르게 문장을 수정하는 것입니다. 답변하는 동안 단계별로 생각하고 답변을 정당화하세요. 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '왜 개인 사생활을 방송으로 보여주고 싶은지 이해도 안가지만 &location&식 프로포즈란 무슨 자로 잰 든 무릎 꿇고 반지 내밀고 나랑 결혼해줄래? 가 전부이다.' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의 바르게 수정합니다.\nassistant: 최종 문장: '개인 사생활을 방송으로 보여주는 이유를 이해하기 어렵지만, &location&식 프로포즈라는 것이 대부분 무릎을 꿇고 반지를 내밀며 결혼을 제안하는 것입니다.'\n혐오 맥락이 남아있다고 판단하며 다음 대항표현을 생성합니다. 대항표현: 각자의 취향과 선택을 존중하는 것이 중요하며, 개인적인 선택을 일반화하는 것은 옳지 않습니다.\nHuman: {x['input2']} 다음은 비속어나 혐오 표현이 포함되어 있는지 확인할 문장입니다. 문장: '{x['input']}' 비속어나 혐오표현이 포함되어 있다면 '혐오'로 판단하고 아니라면 '비혐오'로 판단합니다. '혐오'라고 판단 했다면 주어진 문장을 올바르고 예의바르게 수정합니다.\nassistant:"
    # template = f"""[INST]
    #     문장에서 사용된 욕설 찾아줘
    #     문장 = {x} [/INST]
    # """

    # q = template
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        generation_config=generation_config,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        #stopping_criteria=stopping_criteria,
        streamer=streamer

    )
    result_str = tokenizer.decode(gened[0])

    start_tag = f"Human: "
    start_index = result_str.find(start_tag)

    if start_index != -1:
        result_str = result_str[start_index + len(start_tag):].strip()
    return result_str.split('Human:')[4].split('assistant:')[-1].split("문장:")[-1]

#여기부터 streamlit 파트

img = Image.open('/content/drive/MyDrive/Colab Notebooks/국립국어원 혐오표현/youtube.png')
st.set_page_config(layout="wide")
empty1,con1,empty2 = st.columns([0.3,1.0,0.3])
empty1,con6,empty2 = st.columns([0.3,1.0,0.3])
empty1,con5,empty2=st.columns([0.3,0.5,0.3])
empty1,con4,empty2=st.columns([1.02,0.2,1.0])
empyt1,con2,con3,empty2 = st.columns([0.1,0.6,0.6,0.1])

def scroll(js_path:str='main',
           behavior:str='smooth',
           block:str='center',
           inline:str='center'
           ) -> None:

    js = f'''
        <script>
            const element = window.parent.{js_path};
            element.scrollIntoView({{
                behavior: "{behavior}",
                block: "{block}",
                inline: "{inline}"}});
        </script>
        '''
    st.components.v1.html(js)


def main():

    st.markdown(  """
    <style>ㅇ
    .css-16idsys.e16nr0p34 {
        background-color: green;
    }
    </style>
    """, unsafe_allow_html=True)


    with st.sidebar:
        choose = option_menu("App Gallery", ["About", "Youtube Video", "Comments"],
                            icons=['house', 'camera fill', 'kanban'],
                            menu_icon="app-indicator", default_index=0,
                            styles={
            "container": {"padding": "5!important", "background-color": "#17202A"},
            "icon": {"color": "orange", "font-size": "25px"},
            "nav-link": {"font-size": "16px", "text-align": "left", "margin":"0px", "--hover-color": "#eee"},
            "nav-link-selected": {"background-color": "#02ab21"},
        }
        )

        if choose == "About":
            js_path = 'document.querySelector("#root > div:nth-child(1) > div.withScreencast > div > div > div > section.main.st-emotion-cache-uf99v8.ea3mdgi3 > div.block-container.st-emotion-cache-z5fcl4.ea3mdgi2 > div > div > div > div:nth-child(1) > div.st-emotion-cache-16gvjpr.e1f1d6gn3 > div > div > div > div:nth-child(2)")'
            scroll(js_path=js_path)
        if choose == "Youtube Video":
            js_path = 'document.querySelector("#root > div:nth-child(1) > div.withScreencast > div > div > div > section.main.st-emotion-cache-uf99v8.ea3mdgi3 > div.block-container.st-emotion-cache-z5fcl4.ea3mdgi2 > div > div > div > div:nth-child(2) > div.st-emotion-cache-16gvjpr.e1f1d6gn3")'
            scroll(js_path=js_path)
        if choose == "Comments":
            js_path = 'document.querySelector("#root > div:nth-child(1) > div.withScreencast > div > div > div > section.main.st-emotion-cache-uf99v8.ea3mdgi3 > div.block-container.st-emotion-cache-z5fcl4.ea3mdgi2 > div > div > div > div:nth-child(5) > div:nth-child(2) > div > div > div > div:nth-child(1)")'
            scroll(js_path=js_path)


    with empty1 :
        empty()
    with con1:
        st.image(img, width=80)
        st.markdown("<h1 style='text-align: center; color: red;'>Youtube Comment Converter</h1>", unsafe_allow_html=True)

        # 유튜브 주소 입력받기
        user_input = st.text_input("Enter youtube address:")

    with con6:
        st_player(user_input)


    with con5:
        st.markdown("<h1 style='text-align: center; color: white; font-size: 24px;'>Convert Button</h1>", unsafe_allow_html=True)
    #버튼을 누르면 변환되게끔

    with con2:
        #이 제목 아래에 원래 댓글들 보여준다
        st.markdown("<h1 style='text-align: center; color: white;'>Original Comment</h1>", unsafe_allow_html=True)

        # 입력받은 텍스트가 비어있지 않은 경우에만 변환 후 표시
        if user_input:
            downloader = YoutubeCommentDownloader()
            comments = downloader.get_comments_from_url(user_input, sort_by=SORT_BY_RECENT)
            for i, comment in islice(enumerate(comments, 1), 20):
                st.write(f"{i}:  {comment['text']}")

    with con4:
        if st.button("Convert", key="centered_button", help="Click me!"):
            con = st.container()
            # 버튼이 클릭되었을 때 수행할 작업 추가
            st.write("Button clicked!")

            with con3:
                st.markdown("<h1 style='text-align: center; color: white;'>Converted Comment</h1>", unsafe_allow_html=True)
                comments = downloader.get_comments_from_url(user_input, sort_by=SORT_BY_RECENT)
                for i, comment in islice(enumerate(comments, 1), 20):
                    #input_ids = tokenizer.encode(comment['text'], return_tensors="pt")
                    input_ids = comment['text']
                    if len(input_ids[0]) > 70:
                        input_ids = input_ids[:, :70]

                    judgement_text = gen(input_ids)
                    if judgement_text == "비혐오":
                      generated_text = input_ids
                      final_text = mask_bad_words_optimized(generated_text,bad_words)

                    else:
                      generated_text = gen2(input_ids)
                      final_text = mask_bad_words_optimized(generated_text,bad_words)


                    # 생성된 텍스트 출력
                    st.write(f"{i}: {judgement_text}댓글입니다 ==> 최종 문장: {final_text}")

if __name__ == "__main__":
    main()

Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.125.249.198


In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501

npx: installed 22 in 2.265s
your url is: https://cool-women-happen.loca.lt
^C
